## **IMPORT**

In [ ]:
!pip install rrcf

  Preparing metadata (setup.py) ... done
  Created wheel for rrcf: filename=rrcf-0.4.4-py3-none-any.whl size=10601 sha256=b093777374ac88463421ac81dc4efff9d839c0196a848722b29ae4171184b4f3
  Stored in directory: /root/.cache/pip/wheels/51/d4/43/6ac8270752946ee2223022ce0ed0d66c9455d000e1facb53a6
Successfully built rrcf


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from rrcf import RCTree, rrcf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

## **DATA CLEANING & DATA IMPUTATION**

**FUCNTION THAT IMPUTES VALUES FOR COUNTRIES WITH MISSING YEARS**

Generates values for the missing years between 1900-2020 inclusive for a single country's dataset

**Inputs**: incomplete data set, name of country, code of country, and name of the colunm data to complete

**Outputs**: complete dataset (missing values are imputed)

In [ ]:
def fill_missing(country_missing, entity, region,code,what_is_missing):
    # Filter rows between years 1990 and 2021

    print("entity :",entity, "    code :",code)
    years_range = range(1990, 2021)
    filtered_years = country_missing[country_missing['Year'].isin(years_range)].sort_values('Year')
    print("Filtered years:",len(filtered_years))

    mean1 = filtered_years.loc[((country_missing['Year']>=1990) & (country_missing['Year']<2000) ),what_is_missing].mean()
    mean2 = filtered_years.loc[((country_missing['Year']>=2000) & (country_missing['Year']<2010) ),what_is_missing].mean()
    mean3 = filtered_years.loc[((country_missing['Year']>=2010) & (country_missing['Year']<=2020) ),what_is_missing].mean()
    # Create a DataFrame containing missing years

    if np.isnan(mean1):
        mean1=0
    if np.isnan(mean2):
        mean2=0
    if np.isnan(mean3):
        mean3=0
    totalMean = (mean1+mean2+mean3)/3
    if mean1==0:
        mean1=totalMean
    if mean2==0:
        mean2=totalMean
    if mean3==0:
        mean3=totalMean

    missing_values = pd.DataFrame({'Year': [year for year in years_range if year not in filtered_years['Year'].tolist()]})
    print("Missing years:",len(missing_values))
    missing_values['Entity']=entity
    missing_values['Code']=code
    missing_values['region']=region
    print("Edited mean1 :",mean1, "    mean2 :",mean2,"    mean3 :",mean3)
    # Reindex DataFrame with the column moved to the desired position
    new_columns = ['Entity','Code','region', 'Year']
    missing_values = missing_values.reindex(columns=new_columns)

    # Fill missing GDP values based on closest available values
    for index, row in missing_values.iterrows():

        prev_year=row['Year']
        prev_value=0
        next_year=row['Year']
        next_value=0
        if len(filtered_years[filtered_years['Year'] < row['Year']])>0:
            prev_year = filtered_years[filtered_years['Year'] < row['Year']].iloc[-1]['Year']
            prev_value = filtered_years[filtered_years['Year'] == prev_year][what_is_missing].values[0]

        else:
            prev_year = row['Year']
            prev_value =0


        if len(filtered_years[filtered_years['Year'] > row['Year']])>0:
            next_year = filtered_years[filtered_years['Year'] > row['Year']].iloc[0]['Year']

            next_value = filtered_years[filtered_years['Year'] == next_year][what_is_missing].values[0]

        else:
            next_year = row['Year']
            next_value =0


        # Calculate the GDP difference and distribute it among missing years
        value_difference = abs(next_value - prev_value)
        num_years_difference = abs(next_year - prev_year)
        if num_years_difference == 0:
            num_years_difference=1
        value_per_year_difference = value_difference / num_years_difference

        missing_values.loc[index, what_is_missing] = prev_value + ((row['Year'] - prev_year) * value_per_year_difference)

    # Concatenate the filtered DataFrame and missing_years DataFrame
    result_missing = pd.concat([filtered_years, missing_values]).sort_values('Year')
    print("length:",len(result_missing))


    result_missing.loc[((result_missing['Year']>=1990) & (result_missing['Year']<2000) &  (result_missing[what_is_missing]==0)),what_is_missing]=mean1
    result_missing.loc[((result_missing['Year']>=2000) & (result_missing['Year']<2010) &  (result_missing[what_is_missing]==0)),what_is_missing]=mean2
    result_missing.loc[((result_missing['Year']>=2010) & (result_missing['Year']<=2020) &  (result_missing[what_is_missing]==0)),what_is_missing]=mean3
    return result_missing


**FUNCTION THAT ADDS REGION FEATURE TO A DATASET**

Assigns a fifth column to the dataset of each region (according to The World Bank) based on the country of each row

**inputs**: orginal dataset

**outputs**: dataset with regions added

In [ ]:
def assign_regions(national_countries):
    regions = pd.read_csv("/content/world-regions-according-to-the-world-bank.csv")
    national_countries['region']='blank'
    all_countries = national_countries['Code'].unique()
    for code in all_countries:
        region_country = regions.loc[regions['Code']==code]

        if len(region_country) > 0:

            national_countries.loc[national_countries['Code']==code, 'region']=region_country['World Region according to the World Bank'].iloc[0]
    return national_countries

**FUNCTION THAT IMPUTES VALUES FOR MISSING COUNTRIES BASED ON REGION MEAN**

Creates new rows for any country misssing in the orignal dataset based on the mean of the region and the year of each row

**inputs**: dataset with missing countries, missing vlaues column,

**outputs**: completed dataset with every country

In [ ]:
def filling_country_by_region(totaldata, forth_column):
    world_regions = pd.read_csv("/content/world-regions-according-to-the-world-bank.csv")
    all_countries = world_regions['Code'].unique()
    for country in all_countries:
        if len(totaldata.loc[totaldata['Code']==country])==0 and len(world_regions.loc[world_regions['Code']==country])>0:
            print("country:",country ,"  ",world_regions.loc[world_regions['Code']==country])
            row = world_regions.loc[world_regions['Code']==country].head(1)
            entity = row['Entity'].values[0]
            region_v = row['World Region according to the World Bank'].values
            region = region_v[0]
            for year in  range(1990, 2021):
                test1 = totaldata.loc[totaldata['Year']==year]
                test2 = totaldata.loc[totaldata['region']==region]
                mean = totaldata.loc[(totaldata['Year']==year)&(totaldata['region']==region), forth_column].mean()

                data = {'Entity': [entity], 'Code': [country], 'region': [region], 'Year': [year], forth_column: [mean]}
                new_row = pd.DataFrame(data)
                totaldata = pd.concat([totaldata, new_row]).sort_values('Year')

    return totaldata

FUNCTION THAT REMOVES OUTLIERS

takes a dataset and removes outliers, nan and negative values changed to 0 and other outliers are completly removed uses Robust Random Cut Forest (based on chat gpt)

inputs: orginal dataset, missing values column

output:the orginal dataset with outliers removed

In [ ]:
def check_for_outlier_values(country,what_is_missing):
    for index, row in country.iterrows():
        if np.isnan(row[what_is_missing]) or row[what_is_missing] <0:
            country.loc[index, what_is_missing]=0
            row[what_is_missing] = 0
    data_array = country[what_is_missing].values[:, None]
    num_trees = 50
    forest = []
    for _ in range(num_trees):
        tree = RCTree()
        forest.append(tree)
    lenght=data_array.shape[0]
    for i in range(data_array.shape[0]):
        point = data_array[i]
        for tree in forest:
            tree.insert_point(point, index=i)
    avg_codisp = np.zeros(data_array.shape[0])
    index = np.zeros(data_array.shape[0])
    for tree in forest:
        for i in range(data_array.shape[0]):
            codisp = np.array(tree.codisp(i))
            avg_codisp += codisp
        index += 1
    avg_codisp /= index
    threshold = np.percentile(avg_codisp, 95)
    outliers = country[avg_codisp > threshold]
    if len(outliers) >0:
        print("Outliers:")
        print(outliers)
        print(len(country))
        country = country.drop(outliers, axis = 0).reset_index(drop=True)
        print(len(country))
    return country

**FUNCTION THAT GENERATES IMPUTED AND CLEANED DATASETS**

takes a dataset and fills in the missing values for each country and enusures rows and values are created for any countries missing in the orignal dataset

**inputs**: orginal dataset, missing values column, the filename it wants to save the new dataset as

In [ ]:
def complete_dateset(global_info,forth_column_name,save_file_name):
    complete_info = pd.DataFrame(columns=['Entity','Code','region', 'Year',forth_column_name])
    global_info = assign_regions(global_info)
    all_countries = global_info['Code'].unique()
    print('all countries', len(all_countries))
    for country in all_countries:
        country_info = global_info.loc[global_info['Code']==country]
        if len(country_info)>0:
            years_range = range(1990, 2021)
            country_info = country_info[country_info['Year'].isin(years_range)].sort_values('Year')
            country_info = check_for_outlier_values(country_info,forth_column_name)
            missing_info = fill_missing(country_info,country_info['Entity'].iloc[0] , country_info['region'].iloc[0] ,country,forth_column_name)
            complete_info = pd.concat([complete_info, missing_info]).sort_values('Code')
    print('all countries', len(complete_info['Code'].unique()))
    complete_info = filling_country_by_region(complete_info, forth_column_name)
    file_path = "/content/output/"+save_file_name+".csv"
    complete_info = complete_info.sort_values('Code')
    print('all countries', len(complete_info['Code'].unique()))
    complete_info.to_csv(file_path, index=False)

### **CLEANING EACH DATASET INDIVIDUALLY**

**CALLING THE ABOVE FUNCTIONS TO CLEAN/IMPUTE EACH DATASET**

Taking each original dataset, cleaning it, replacing missing values and
saving the completed version

In [ ]:
national_gdp= pd.read_csv("/content/national-gdp-constant-usd-wb.csv")
national_gdp.rename(columns = {'GDP (constant 2015 US$)':'GDP'}, inplace = True)sni
complete_dateset(national_gdp,'GDP','complete_gdp')


all countries 212
entity : Afghanistan     code : AFG
Filtered years: 19
Missing years: 12
Edited mean1 : 9859905988.636364     mean2 : 9984893875.0     mean3 : 19594824090.909092
length: 31
entity : Albania     code : ALB
Filtered years: 31
Missing years: 0
Edited mean1 : 4586803240.0     mean2 : 7974835000.0     mean3 : 11597895181.818182
length: 31
entity : Algeria     code : DZA
Filtered years: 31
Missing years: 0
Edited mean1 : 83400954500.0     mean2 : 118284566000.0     mean3 : 162031650000.0
length: 31
entity : American Samoa     code : ASM
Filtered years: 19
Missing years: 12
Edited mean1 : 462304133.0681818     mean2 : 732593883.75     mean3 : 654318515.4545455
length: 31
entity : Andorra     code : AND
Filtered years: 31
Missing years: 0
Edited mean1 : 1827964890.0     mean2 : 2883315110.0     mean3 : 2844400300.0
length: 31
entity : Angola     code : AGO
Filtered years: 31
Missing years: 0
Edited mean1 : 24999286600.0     mean2 : 46549870100.0     mean3 : 81097054909.09091


In [ ]:
national_population= pd.read_csv("/content/population.csv")
national_population.rename(columns = {'Population (historical estimates)':'Population'}, inplace = True)
complete_dateset(national_population,'Population','complete_population')

In [ ]:
national_birth_rate= pd.read_csv("/content/crude-birth-rate.csv")
national_birth_rate.rename(columns = {'Birth rate - Sex: all - Age: all - Variant: estimates':'Birth rate(pre 1000)'}, inplace = True)
complete_dateset(national_birth_rate,'Birth rate(pre 1000)','complete_birth_rate')

In [ ]:
national_fertility_rate= pd.read_csv("/content/children-born-per-woman.csv")
national_fertility_rate.rename(columns = {'Fertility rate (Select Gapminder, v12) (2017)':'Fertility rate'}, inplace = True)
complete_dateset(national_fertility_rate,'Fertility rate','complete_fertility_rate')


In [ ]:
national_gdp_per_capita = pd.read_csv("/content/gdp-per-capita-worldbank.csv")
national_gdp_per_capita.rename(columns = {'GDP per capita, PPP (constant 2017 international $)':'GDP per capita'}, inplace = True)
complete_dateset(national_gdp_per_capita,'GDP per capita','complete_gdp_per_capita')


In [ ]:
national_mortality_rate = pd.read_csv("/content/age-standardized-deaths-from-all-causes.csv")
national_mortality_rate.rename(columns = {'Deaths - All causes - Sex: Both - Age: Age-standardized (Rate)':'Death rate'}, inplace = True)
complete_dateset(national_mortality_rate,'Death rate','complete_mortality_rate')

In [ ]:
national_child_mortality = pd.read_csv("/content/fertility-vs-child-mortality.csv")
national_child_mortality.drop(columns= ["Fertility rate - Sex: all - Age: all - Variant: estimates", "Population (historical estimates)","Continent"], inplace = True)
national_child_mortality.rename(columns = {'Child mortality rate - Sex: all - Age: 0-4 - Variant: estimates':'Child mortality rate %'}, inplace = True)
complete_dateset(national_child_mortality,'Child mortality rate %','complete_child_mortality')

In [ ]:
national_life_expectancy = pd.read_csv("/content/life-expectancy.csv")
national_life_expectancy.rename(columns = {'Period life expectancy at birth - Sex: all - Age: 0':'Period life expectancy'}, inplace = True)
complete_dateset(national_life_expectancy,'Period life expectancy','complete_life_expectancy')


In [ ]:
national_urban_population= pd.read_csv("/content/urban-and-rural-population.csv")
national_urban_population.drop(columns= ["Rural population"], inplace = True)
complete_dateset(national_urban_population,'Urban population','complete_urban_population')

In [ ]:
national_rural_population= pd.read_csv("/content/urban-and-rural-population.csv")
national_rural_population.drop(columns= ["Urban population"], inplace = True)
complete_dateset(national_rural_population,'Rural population','complete_rural_population')


In [ ]:
national_female_labor_force= pd.read_csv("/content/female-labor-force-participation-rates.csv")
national_female_labor_force.rename(columns = {'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)':'female labor force participation rate'}, inplace = True)
complete_dateset(national_female_labor_force,'female labor force participation rate','complete_female_labor_force')


In [ ]:
national_schooling= pd.read_csv("/content/mean-years-of-schooling-long-run.csv")
national_schooling.rename(columns = {'Combined - average years of education for 15-64 years male and female youth and adults':'Average years of education'}, inplace = True)
complete_dateset(national_schooling,'Average years of education','complete_schooling')


In [ ]:
national_consumer= pd.read_csv("/content/consumer-price-index.csv")
national_consumer.rename(columns = {'Consumer price index (2010 = 100)':'Consumer price index'}, inplace = True)
complete_dateset(national_consumer,'Consumer price index','complete_consumer')


**DATA INTEGRATION**

integrate (200) datasets for individual country religion statistics into a single dataset for all countries

In [ ]:
complete_religious= pd.DataFrame(columns=['Entity','Code','Year','Atheists'])

national_gdp= pd.read_csv("/content/output/complete_gdp.csv")
all_countries = national_gdp['Entity'].unique()
print('all countries', len(all_countries))
for country in all_countries:
  database_name= '/content/religion_data_sets/'+country + '_ Largest Religious Groups  (1900 - 2050) (World Religion Database, 2020).csv'
  if country !='World' and type(country)==str and os.path.exists(database_name):
    #print(type(country)," ",country)
    national_religion= pd.read_csv(database_name)
    national_religion.rename(columns = {'YEAR':'Year'}, inplace = True)
    if "R704" in national_religion.columns:
        national_religion.rename(columns = {'R704':'Atheists'}, inplace = True)
    else:
        national_religion['Atheists']=0
    columns_religion = national_religion.loc[:, (national_religion.columns != 'Atheists' )&(national_religion.columns != 'Year')]
    national_religion.drop(columns=columns_religion, inplace = True)

    national_religion["Entity"]= country
    print("Entity", national_religion["Entity"].iloc[0])
    code = national_gdp.loc[national_gdp['Entity']==country, 'Code']
    national_religion["Code"]= code.iloc[0]
    national_religion.rename(columns = {'YEAR':'Year'}, inplace = True)


    new_columns = ['Entity','Code','Year','Atheists']
    national_religion = national_religion.reindex(columns=new_columns)
    complete_religious= pd.concat([complete_religious, national_religion]).sort_values('Code')

complete_religious.to_csv('/content/output/complete_religious.csv', index=False)

In [ ]:
national_atheists= pd.read_csv("/content/output/complete_religious.csv")
national_atheists.rename(columns = {'Atheists':'Atheists rate'}, inplace = True)
complete_dateset(national_atheists,'Atheists rate','complete_atheists')


In [ ]:

# # transform columns in the education dataframe to facilitate entity linkage
# # Split the column into multiple columns
# education_df[['Entity', 'Code', 'Year', 'avg_years_of_schooling']] = education_df['Entity;Code;Year;avg_years_of_schooling'].str.split(';', expand=True)

# # Drop the original column
# education_df = education_df.drop('Entity;Code;Year;avg_years_of_schooling', axis=1)

# #convert the type of Year column to int
# education_df['Year'] = education_df['Year'].astype(int)

**READ COMPLETED DATASETS**

these datasets have been cleaned and imputed

before running this code create a folder called **"output"**. The files will be saved to that folder.

In [ ]:
complete_athetits= pd.read_csv("/content/output/complete_atheists.csv")
complete_birth_rate= pd.read_csv("/content/output/complete_birth_rate.csv")
complete_child_mortality= pd.read_csv("/content/output/complete_child_mortality.csv")
complete_consumer= pd.read_csv("/content/output/complete_consumer.csv")
complete_female_labor_force= pd.read_csv("/content/output/complete_female_labor_force.csv")
complete_fertility_rate= pd.read_csv("/content/output/complete_fertility_rate.csv")
complete_gdp= pd.read_csv("/content/output/complete_gdp.csv")
complete_gdp_per_capita= pd.read_csv("/content/output/complete_gdp_per_capita.csv")
complete_life_expectancy= pd.read_csv("/content/output/complete_life_expectancy.csv")
complete_mortality_rate= pd.read_csv("/content/output/complete_mortality_rate.csv")
complete_population= pd.read_csv("/content/output/complete_population.csv")
complete_rural_population= pd.read_csv("/content/output/complete_rural_population.csv")
complete_schooling= pd.read_csv("/content/output/complete_schooling.csv")
complete_urban_population= pd.read_csv("/content/output/complete_urban_population.csv")



**DATA INTEGRATION**

integrate all individual cleaned/imputed datasets into a single complete dataset

before running this code create a folder called "**final**". The files will be saved to that folder.

In [ ]:
merge1_df = pd.merge(complete_athetits, complete_birth_rate, on=['Code', 'Year','Entity', 'region'], how='inner')
merge2_df = pd.merge(merge1_df, complete_child_mortality,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge3_df = pd.merge(merge2_df, complete_consumer,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge4_df = pd.merge(merge3_df, complete_female_labor_force,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge5_df = pd.merge(merge4_df, complete_fertility_rate,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge6_df = pd.merge(merge5_df, complete_gdp,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge7_df = pd.merge(merge6_df, complete_gdp_per_capita,  on=['Code', 'Year','Entity', 'region'], how='inner')

merge8_df = pd.merge(merge7_df, complete_life_expectancy,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge9_df = pd.merge(merge8_df, complete_mortality_rate,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge10_df = pd.merge(merge9_df, complete_population,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge11_df = pd.merge(merge10_df, complete_rural_population,  on=['Code', 'Year','Entity', 'region'], how='inner')
merge12_df = pd.merge(merge11_df, complete_schooling,  on=['Code', 'Year','Entity', 'region'], how='inner')
mergecomplete_df = pd.merge(merge12_df, complete_urban_population,  on=['Code', 'Year','Entity', 'region'], how='inner')
mergecomplete_df.drop(columns= ["region"], inplace = True)
mergecomplete_df.to_csv('/content/final/complete_dataset.csv', index=False)



# **TRANSFORMATION**

**GENERATE ONE HOT ENCODING FOR TEXT COLUMNS**

In [ ]:
test_df = mergecomplete_df

In [ ]:
def generate_one_hot_encode_dataframe(dataframe, column_name):
  text_column_name = [column_name]
  extracted_text_column = dataframe[text_column_name]

  encoder = OneHotEncoder(sparse = False)
  transformed_text_column = encoder.fit_transform(extracted_text_column)
  encoded_df = pd.DataFrame(transformed_text_column, columns=encoder.get_feature_names_out(text_column_name))

  dataframe.drop(column_name, axis=1, inplace=True)

  one_hot_encoded_df = pd.concat([dataframe,encoded_df], axis = 1)

  return one_hot_encoded_df

save dataset as csv

In [ ]:
encoded_dataframe = generate_one_hot_encode_dataframe(test_df,'Entity')

encoded_dataframe.to_csv('/content/final/one_hot_encoded_data.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


**SCALE ABSOLUTE POPULATIONS VALUES TO RATIO VLAUES**

In [ ]:
def scale_population_columns(dataframe, list_of_column_names):
  for i in range(len(list_of_column_names)):
    dataframe[list_of_column_names[i]] = dataframe[list_of_column_names[i]]/dataframe['Population']
  return dataframe

save dataset as csv

In [ ]:
list_of_column_names_1 = ['Urban population', 'Rural population']
scaled_population_dataframe = scale_population_columns(encoded_dataframe, list_of_column_names_1)

scaled_population_dataframe.to_csv('/content/final/scaled_population_data.csv', index=False)

**SCALE PERCENTAGE TO [0-1] SCALE**

In [ ]:
def scale_percentage_columns(dataframe, list_of_column_names):
  for i in range(len(list_of_column_names)):
    dataframe[list_of_column_names[i]] = dataframe[list_of_column_names[i]]/100
  return dataframe

save dataset as csv

In [ ]:
list_of_column_names_2 = ['Atheists rate', 'female labor force participation rate', 'Child mortality rate %']
scaled_percentage_dataframe = scale_percentage_columns(encoded_dataframe, list_of_column_names_2)

scaled_percentage_dataframe.to_csv('/content/final/scaled_percentage_data.csv', index=False)

**SCALE MIN-MAX**

In [ ]:
def min_max_scale_columns(dataframe,list_of_column_names):
  min_max_scaler = MinMaxScaler()

  dataframe[list_of_column_names] = min_max_scaler.fit_transform(dataframe[list_of_column_names])
  return dataframe

save dataset as csv

In [ ]:
list_of_column_names_3 = ['Year','GDP','GDP per capita','Consumer price index', 'Period life expectancy', 'Death rate', 'Population','Average years of education']
min_max_scaled_dataframe = min_max_scale_columns(encoded_dataframe, list_of_column_names_3)

min_max_scaled_dataframe.to_csv('/content/final/min_max_scaled_data.csv', index=False)

**drop unwanted columns**

In [ ]:
columns_to_drop = ['Code','Fertility rate']
min_max_scaled_dataframe = min_max_scaled_dataframe.drop(columns = columns_to_drop)

min_max_scaled_dataframe.to_csv('/content/final/fully_transformed_dataset.csv', index=False)

**impute missing values in child mortality column**

missed some values during imputation of child mortality dataset*

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

final_df = pd.DataFrame(imputer.fit_transform(min_max_scaled_dataframe), columns=min_max_scaled_dataframe.columns)

**put birthrate column as the last column of the dataframe**

In [ ]:
birthrate_column= final_df['Birth rate(pre 1000)']

In [ ]:
final_df = final_df.drop('Birth rate(pre 1000)',axis=1)
final_df['Birth rate(per 1000)'] = birthrate_column

In [ ]:
final_df.to_csv('/content/final/transformed_dataset.csv', index=False)